In [1]:
import os
os.environ["ZFIT_DISABLE_TF_WARNINGS"] = "1"
import zfit
import matplotlib.pyplot as plt
import numpy as np
from hepstats.hypotests.parameters import POI
from hepstats.hypotests.calculators import AsymptoticCalculator, FrequentistCalculator
from hepstats.hypotests import Discovery
import json

/work/gallim/anaconda3/envs/stats-studies/lib/python3.9/site-packages/uproot/reading.py:186: FutureWarning: XRootD 5.1.1 is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)


In [2]:
# Use as input the same json file already created for the pyhf part
info = json.load(open("datacard_part1.json"))

This is a simple example of counting experiment.

First of all, let's take a look at the uncertainties we have: for all the nuisance parameters they are **normsys**, while the POI ($mu$, or $r$ in Combine) is of the **normfactor** type. The meaning is explained in the HistFactory document.

As can be seen in the PyHF documentation and tutorials, **normsys** indicates a *constrained* scaling parameter controlled by a single nuisance parameter, while **normfactor** is the same, but *unconstrained*.

In [3]:
# Good starting point: https://github.com/scikit-hep/hepstats/blob/master/notebooks/hypotests/counting.ipynb

signal = {
    "bbHtautau": {
        "unc": {}
    }
}
bkgs = {
    "ttbar": {
        "unc": {}
    }, 
    "diboson": {
        "unc": {}
    }, 
    "Ztautau": {
        "unc": {}
    }, 
    "jetFakes": {
        "unc": {}
    }
}

for sample in info["channels"][0]["samples"]:
    if sample["name"] in signal:
        signal[sample["name"]]["value"] = sample["data"][0]
        for modifier in sample["modifiers"]:
            if modifier["data"]:
                signal[sample["name"]]["unc"][modifier["name"]] = modifier["data"]["hi"] * sample["data"][0] - sample["data"][0]
    elif sample["name"] in bkgs:
        bkgs[sample["name"]]["value"] = sample["data"][0]
        for modifier in sample["modifiers"]:
            if modifier["data"]:
                bkgs[sample["name"]]["unc"][modifier["name"]] = modifier["data"]["hi"] * sample["data"][0] - sample["data"][0]
        
obs = info["observations"][0]["data"][0]

print(signal)
print(bkgs)

{'bbHtautau': {'unc': {'CMS_eff_b': 0.014221280000000003, 'CMS_eff_t': 0.08532768000000013, 'CMS_eff_t_highpt': 0.07110640000000001, 'acceptance_bbH': 0.03555320000000006}, 'value': 0.711064}}
{'ttbar': {'unc': {'CMS_eff_b': 0.08876060000000052, 'CMS_eff_t': 0.5325636000000005, 'CMS_eff_t_highpt': 0.44380300000000084, 'acceptance_ttbar': 0.022190149999999242}, 'value': 4.43803}, 'diboson': {'unc': {'CMS_eff_b': 0.06366180000000021, 'CMS_eff_t': 0.3819708000000004, 'CMS_eff_t_highpt': 0.3183090000000002, 'xsec_diboson': 0.1591545000000001}, 'value': 3.18309}, 'Ztautau': {'unc': {'CMS_eff_b': 0.0756079999999999, 'CMS_eff_t': 0.4536480000000003, 'CMS_eff_t_highpt': 0.3780400000000004, 'acceptance_Ztautau': 0.3024320000000005}, 'value': 3.7804}, 'jetFakes': {'unc': {'norm_jetFakes': 0.32679199999999997}, 'value': 1.63396}}


In [4]:
# How can constraints be implemented?

In [5]:
N_sig = zfit.Parameter("N_sig", signal["bbHtautau"]["value"], 0, 10)
N_ttbar = zfit.Parameter("N_ttbar", bkgs["ttbar"]["value"], 0, 10)
N_diboson = zfit.Parameter("N_diboson", bkgs["diboson"]["value"], 0, 10)
N_Ztautau = zfit.Parameter("N_Ztautau", bkgs["Ztautau"]["value"], 0, 10)
N_jetFakes = zfit.Parameter("N_jetFakes", bkgs["jetFakes"]["value"], 0, 10)

N_obs = zfit.ComposedParameter(
    "N_obs", 
    lambda sig, b1, b2, b3, b4: sig + b1 + b2 + b3 + b4,
    params = [N_sig, N_ttbar, N_diboson, N_Ztautau, N_jetFakes]
)

In [6]:
observable = zfit.Space("N", limits=(0, 20))

In [7]:
data = zfit.data.Data.from_numpy(obs=observable, array=np.array([obs]))

In [8]:
# signal pdf and constraints definition

#pdf_sig = zfit.pdf.Poisson(obs=observable, lamb=N_sig)
#sig_constraint = []
#for constr in signal["bbHtautau"]["unc"].values():
    #sig_constraint.append(zfit.constraint.GaussianConstraint(N_sig, observation=signal["bbHtautau"]["value"], uncertainty=constr))

In [9]:
#pdf_ttbar = zfit.pdf.Poisson(obs=observable, lamb=N_ttbar)
#pdf_diboson = zfit.pdf.Poisson(obs=observable, lamb=N_diboson)
#pdf_Ztautau = zfit.pdf.Poisson(obs=observable, lamb=N_Ztautau)
#pdf_jetFakes = zfit.pdf.Poisson(obs=observable, lamb=N_jetFakes)

#model = zfit.pdf.ProductPDF(pdfs=[pdf_sig, pdf_ttbar, pdf_diboson, pdf_Ztautau, pdf_jetFakes])

In [10]:
model = zfit.pdf.Poisson(obs=observable, lamb=N_obs)

In [11]:
# likelihood function
nll = zfit.loss.UnbinnedNLL(model=model, data=data)

# Instantiate a minuit minimizer
minimizer = zfit.minimize.Minuit()

# minimisation of the loss function
minimum = minimizer.minimize(loss=nll)
print(minimum)

FitResult of
<UnbinnedNLL model=[<zfit.<class 'zfit.models.dist_tfp.Poisson'>  params=[N_obs]] data=[<zfit.core.data.Data object at 0x7f20104a6d30>] constraints=[]> 
with
<Minuit Minuit tol=0.001>

╒═════════╤═════════════╤══════════════════╤═════════╤═════════════╕
│ valid   │ converged   │ param at limit   │ edm     │ min value   │
╞═════════╪═════════════╪══════════════════╪═════════╪═════════════╡
│ True    │ True        │ False            │ 1.4e-05 │ 999.5       │
╘═════════╧═════════════╧══════════════════╧═════════╧═════════════╛

Parameters
name          value    at limit
----------  -------  ----------
N_sig        0.5091       False
N_ttbar       3.201       False
N_diboson     2.339       False
N_Ztautau     2.765       False
N_jetFakes    1.188       False
